# Creating AI Agents With LangChain

The core idea of agents is to use a language model to choose a sequence of actions to take. In chains, a sequence of actions is hardcoded (in code). In agents, a language model is used as a reasoning engine to determine which actions to take and in which order.

The key components of an Agent include a `Language Model` (the cognitive center), `Tools` (for interacting with external systems), and an `Agent Executor` (the runtime environment).

LangChain offers a variety of pre-built tools, such as Wikipedia, Calculator, and Search engines, and also allows developers to create custom tools.

The framework supports the creation of simple and complex agents, from basic search assistants to sophisticated AI systems that interact with multiple data sources and APIs.

Agents can be customized by creating and integrating custom tools, allowing for flexibility in addressing various real-world problems. Tools provide communication between Agents, chains, chat models, and other external systems and data sources. If an LLM is given a list of available tools and a prompt, it can ask to have one or more tools invoked with the proper inputs.

## Configuring the environment and installing 

First create a new Environment using the menu. Add the following information in the new environment:

```
Environment Variables Set Name : OpenAI

NAME    : OPENAI_API_KEY
VALUEL  : 
```

As the second step, run the following cell to install all the required packages:

In [1]:
!pip install langchain 
!pip install langchain_community
!pip install langchain-openai
!pip install langchain_community
!pip install -qU langchain-community faiss-cpu
!pip install langsmith
!pip install langchain_core

_**After the packages are install you must restart the kernel through: `Run -> Restart Kernel`**_

## First Tool: Retriever Tool

A retriever tool is used to retrieve relevant data from a vector database based on the user input and sent to an LLM as context for the user input. This is done when the source data is large and cannot be send as part of the user input in the prompt. 

In the below example of retriever tool, data from a website https://www.dasa.org (you can choose another one if you prefer) is stored in a vector database, and the retriever tool is used to retrieve relevant information from the vector database to be passed to the LLM as context.

In order to provide the tools to the AI Agent, we first need to create the retriever and then import the tool.

### Preparing the data for the retriever tool

We will use the `WebBaseLoader` object from the `langchain_community.document_loaders` package.

Pass the website url to the `WebBaseLoader` object, such as WebBaseLoader("my_url"), and assign the outcome to a variable called `loader`.

In [3]:
OPENAI_API_KEY = ""

In [17]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://tr.wikipedia.org/wiki/Vim_(metin_d%C3%BCzenleyici)")

docs = loader.load()

Now that we have the content, we will split it into chunks using `RecursiveCharacterTextSplitter`, which should be imported from `langchain_text_splitters`. 

The `RecursiveCharacterTextSplitter` takes a large text and splits it based on a specified chunk size. It does this by using a set of characters. The default characters provided to it are ["\n\n", "\n", " ", ""].

It takes in the large text then tries to split it by the first character \n\n. If the first split by \n\n is still large then it moves to the next character which is \n and tries to split by it. If it is still larger than our specified chunk size it moves to the next character in the set until we get a split that is less than our specified chunk size. 

If you are curious about how it works, you can refer to this useful in-depth resource: dev.to/eteimz/understanding-langchains-recursivecharactertextsplitter-2846

In [18]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()

Now we have the text_splitter object ready, we can call its `split_documents` method passing `docs` as a parameter. The result should be saved into a variable called `documents`.

In [19]:
documents = text_splitter.split_documents(docs)

### Storing the data into a vector database

Our data is ready, stored inside the `documents` object. Now, we need to compute its embeddings (i.e., convert it into numbers), and store the embeddings into a vector database.

We will use the `FAISS` tool to create the vector database. Similar to chromadb that we used previously, Facebook AI Similarity Search (FAISS) is a library for efficient similarity search and clustering of dense vectors. It contains algorithms that search in sets of vectors of any size, up to ones that possibly do not fit in RAM. It also includes supporting code for evaluation and parameter tuning.

For a vector database, we always need an embedding function (responsible for converting the text data into numbers). This time we will use the OpenAIEmbeddings. 

Please execute the following cell, which contains all the necessary code for importing the `FAISS` and the `OpenAIEmbeddings` packages.

In [13]:
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    api_key=OPENAI_API_KEY,
    model="gpt-3.5-turbo",
)

We need to pass the text data (documents), and the embedding function (embeddings) into the from_documents methond of FAISS:

In [20]:
vector = FAISS.from_documents(documents = documents, embedding = embeddings)
retriever = vector.as_retriever() # the retriever will be used to create the tool later

PermissionDeniedError: Error code: 403 - {'error': {'message': 'You are not allowed to generate embeddings from this model', 'type': 'invalid_request_error', 'param': None, 'code': None}}

### Creating the Tool

We will use the `create_retriever_tool` in LangChain to create a tool specifically for retrieving information.

In [21]:
from langchain.tools.retriever import create_retriever_tool

In [22]:
retriever_tool = create_retriever_tool(
    retriever,
    "a title for the tool", # tool name
    "description of the tool") # <<example: This tools is for searching information about X. Use this tool when responding questions about X>>

NameError: name 'retriever' is not defined

Now the first tool is ready. We will use it later within the AI Agent. Next, we will create the second tool below.

## Second Agent: Web Search

Let’s create another tool that can be used to search the Internet. This can be done through the Tavily Search tool for which you need an API that you can get for free at tavily.com.

In [ ]:
import os
os.environ['TAVILY_API_KEY'] = "<<api key goes here>>"

from langchain_community.tools.tavily_search import TavilySearchResults
search = TavilySearchResults()

Now, we have two tools that we can give to the AI Agent. Let us put them in a list.

In [9]:
tools = [retriever_tool, search]

### Create the AI Agent

**Prompt Template for LLM**

We will use langsimith to pull an existing prompt template useful for making an LLM behave as an AI agent.

In [14]:
from langsmith.client import Client, convert_prompt_to_openai_format

# langsmith client
client = Client()

The code below will pull the `"hwchase17/openai-functions-agent"` template:

In [15]:
prompt = client.pull_prompt(prompt_identifier="hwchase17/openai-functions-agent")

Below you can see the structure of the prompt:

In [13]:
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

The placeholder `agent_scratchpad` will hold any intermediate thoughts, calculations, or information that the LLM generates while processing the user's request. This can be useful for debugging or understanding the LLM's reasoning process.

Alternatively, you could just define the following :

```
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(    [
        ("system", "You are a helpful assistant"),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)
```

> **The key point is:** The agent prompt must have an `agent_scratchpad` key that is a MessagesPlaceholder. Intermediate agent actions and tool output messages will be passed in here.

**Creating the LLM**

We will simply use OpenAI to create the LLM for chat:

In [18]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI()

**Creating the Agent**

Next, let us import some libraries from LangChain to create agents and execute agents.

In [17]:
from langchain.agents import AgentExecutor, create_openai_functions_agent

We are now ready to create an AI agent. The AI agent needs an llm, tools and a prompt. For this purpose, make a call to the `create_openai_functions_agent function`, and pass the `llm`, `tools`, and `prompt` as the function parameters.

In [23]:
agent = create_openai_functions_agent(llm, tools=tools, prompt=prompt)

NameError: name 'create_openai_functions_agent' is not defined

To execute an agent, we need to call the `AgentExecutor`. The AgentExecutor is tasked with utilizing the Agent until the final output is attained. Consequently, it employs the Agent to obtain the next action, executes the returned action iteratively, and continues this process until a conclusive answer is generated for the given input.

In [ ]:
agent_executor = AgentExecutor(agent=agent ,tools=tools,verbose=True)

In [ ]:
agent_executor.invoke({"input":"<< A question about the website >>"}, verbouse=True)

Based on the user input, the AI agent decided to use the retriever tool.

Let’s ask some other question not related to the website. This time, the AI agent should decide to use the Tavily Search tool to search the Internet instead of the retriever tool.

In [ ]:
agent_executor.invoke({"input":"<<Your question>>"})